In [ ]:
import numpy as np 
import pandas as pd 
df = pd.read_csv('../input/children5/Child.csv')
df.rename(columns={'emotion\t':'emotion'},inplace=True)
df.to_csv('ChildEmotion1.csv')
df = pd.read_csv('./ChildEmotion1.csv')
df.drop("Unnamed: 0",inplace=True,axis=1)
#df.drop("Usage",inplace=True,axis=1)
df.head()

In [ ]:
X = []
y = []
label_map = ['Neutral','Happy', 'Sad']


In [ ]:
def getData(path):
    sad = 0
    happy = 0
    neutral = 0
    df = pd.read_csv(path)
    
    X = []
    y = []    
    
    for i in range(len(df)):
        if df.iloc[i]['emotion'] != 1:
                
                
            if df.iloc[i]['emotion'] == 3:
                if happy <= 4000:            
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    happy += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 4:
                if sad <= 4000:            
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    sad += 1
                else:
                    pass
                
            
                
            if df.iloc[i]['emotion'] == 6:
                if neutral <= 4000:            
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    neutral += 1
                else:
                    pass

            
            
    return X, y 

In [ ]:
X, y = getData('./ChildEmotion1.csv')
     

In [ ]:
np.unique(y, return_counts=True)
print(y)

In [ ]:
X = np.array(X)/255.0
y = np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
y_o = []
for i in y:
    if i == 6:
        y_o.append(0)
    elif i == 3:
        y_o.append(1)
    elif i == 4:
        y_o.append(2)


In [ ]:
np.unique(y_o, return_counts=True)

In [ ]:
import matplotlib.pyplot as plt
for i in range(5):
    r = np.random.randint(1,17)
    plt.figure()
    plt.imshow(X[r].reshape(48,48), cmap='gray')
    plt.xlabel(label_map[y_o[r]])

In [ ]:
X = X.reshape(len(X), 48, 48, 1)

In [ ]:
from keras.utils import to_categorical
y_new = to_categorical(y_o, num_classes=3)

In [ ]:
len(y), y_new.shape

In [ ]:
y_o[5], y_new[5]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [ ]:
model = Sequential()


input_shape = (48,48,1)


model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(32, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(64, (5, 5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

## (15, 15) --->  30
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X, y_new, epochs=22, batch_size=11, shuffle=True, validation_split=0.2)

In [ ]:
import cv2
test_img = cv2.imread('../input/happy-child/child-14.png', 0)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img, (48,48))
test_img.shape

In [ ]:
plt.imshow(test_img,cmap='gray')

In [ ]:
test_img = test_img.reshape(1,48,48,1)

In [ ]:
model.predict(test_img)

In [ ]:
# label_map = [Neutral','Happy','Sad']